AAAAAAAAA

Imports

In [60]:
import math
from collections import Counter
import re
import pandas as pd


Database:

In [61]:
example_text = "Cats are loved by everyone"

example_text2 = "I love cats so much"

example_text3 = "OMG cats are amazing, I love them"

database = [
    example_text,
    example_text2,
    example_text3
]

example_text4 = "Cats are amazing creatures, enchanting us with their graceful movements and independent personalities. Their soft purrs and gentle nuzzles provide comfort, fostering a unique bond with their human companions. With keen senses, agile leaps, and adorable antics, cats bring joy to homes worldwide. Their mysterious charm captivates hearts, making them truly extraordinary."

TF-IDF

There are multiple ways to calculate the TF(term frequency)

I chose to use the Logarithmically scaled frequency

IDF - inverse document frequency
- looks ar common/uncommon words - corrects for words like as, of, the etc
- minimize the weighting of frequent terms while making infrequent terms have a higher impact.


In [ ]:
def log_scaled_tf(raw_tf):
    # Adding 1 to avoid log(0) issue
    return 1 + math.log(raw_tf) if raw_tf > 0 else 0

def ltfidf(database):
    all_words = [re.findall(r'\b\w+\b', text.lower()) for text in database]
    
    words = [word for sublist in all_words for word in sublist]

    term_counts = Counter(words)
    
    total_terms = len(words)
    
    ltf = {term: log_scaled_tf(count) for term, count in term_counts.items()}
    
    term_document_count = Counter(term for doc in database for term in set(re.findall(r'\b\w+\b', doc.lower())))
    idf = {term: math.log(len(database) / (1 + term_document_count[term])) for term in ltf}

    ltf_idf = {term: ltf[term] * idf[term] for term in ltf}

    return ltf_idf

ltfidf_result = ltfidf(database)

for term, ltf_idf in ltfidf_result.items():
    print(f"Term: {term}, LTFIDF: {ltf_idf:.4f}")

print("==========")


And then I decided that the simple TF should also work

In [ ]:
def tfidf(database):
    all_words = [re.findall(r'\b\w+\b', text.lower()) for text in database]
    
    words = [word for sublist in all_words for word in sublist]

    term_counts = Counter(words)
    
    total_terms = len(words)
    
    #so it's basically how frequent the term is/sum of all terms in the document (example text)

    tf = {term: count / total_terms for term, count in term_counts.items()}
        
    #counts each unique term across all documents in database
    term_document_count = Counter(term for examples in database for term in set(re.findall(r'\b\w+\b', examples.lower())))
    
    #take the log of the number of documents over the set of documents that have the term term
    idf = {term: math.log(len(database) / (1 + term_document_count[term])) for term in tf}

    tf_idf = {term: tf[term] * idf[term] for term in tf}

    return tf_idf

tfidf_result = tfidf(database)

for term, tf_idf in tfidf_result.items():
    print(f"Term: {term}, TFIDF: {tf_idf:.4f}")


And then I discovered that you can do all these using TfidfVectorizer that basically doesn everything you need but in simpler t

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
result = tfidf.fit_transform(database)

print('\nidf values:')
feature_names = tfidf.get_feature_names_out()
for feature, idf_value in zip(feature_names, tfidf.idf_):
    print(feature, ':', idf_value)

In [ ]:
print('\nWord indexes:')
sorted_tfidf_vocabulary = sorted(tfidf.vocabulary_.items(), key=lambda x: x[1])
print(sorted_tfidf_vocabulary)

In [ ]:
print('\ntf-idf value:')
print(result)

#(a, b) = a is document index, b is word index

In [ ]:
print('\ntf-idf values with word indices:')
feature_names = tfidf.get_feature_names_out()

for i, doc in enumerate(result):
    for j, value in zip(doc.indices, doc.data):
        word = feature_names[j]
        print(f"  ({i + 1}, {word}:{j}) {value:.4f}")

In [ ]:
print('\ntf-idf values in matrix form:')
print(result.toarray())

So basically, each row is a document(example text), and each column is a word from the corpus - we have 11 words for now so wooohoooo